**Imports**

In [5]:
import subprocess
import os

**1. Conversions**



In [ ]:
def convert_to_vp8(input_path, output_path="output_vp8.webm"):
    cmd = [
        "ffmpeg",
        "-i", input_path,
        "-c:v", "libvpx",
        "-b:v", "1M",
        "-c:a", "libvorbis",
        output_path
    ]
    subprocess.run(cmd, check=True)
    print(f"VP8 conversion complete: {output_path}")

def convert_to_vp9(input_path, output_path="output_vp9.webm"):
    cmd = [
        "ffmpeg",
        "-i", input_path,
        "-c:v", "libvpx-vp9",
        "-b:v", "1M",
        "-c:a", "libvorbis",
        output_path
    ]
    subprocess.run(cmd, check=True)
    print(f"VP9 conversion complete: {output_path}")

def convert_to_h265(input_path, output_path="output_h265.mp4"):
    cmd = [
        "ffmpeg",
        "-i", input_path,
        "-c:v", "libx265",
        "-crf", "28",
        "-c:a", "aac",
        'b:a', '128k',
        output_path
    ]
    subprocess.run(cmd, check=True)
    print(f"H265 conversion complete: {output_path}")


def convert_to_av1(input_path, output_path="output_av1.mp4"):
    cmd = [
        "ffmpeg",
        "-i", input_path,
        "-c:v", "libaom-av1",
        "-crf", "30",
        'b:v', '0',
        '-strict', 'experimental',
        output_path
    ]
    subprocess.run(cmd, check=True)
    print(f"AV1 conversion complete: {output_path}")

def convert_to_all(input_path):
    convert_to_vp8(input_path)
    convert_to_vp9(input_path)
    convert_to_h265(input_path)
    convert_to_av1(input_path)
    